1. 침수심 csv 에서 구별, 날짜, 침수심(y) 열 데이터를 불러온다.(기준)

In [329]:
import pandas as pd

#침수심_부산_수정.csv 데이터를 불러와서 기준 데이터만 뽑아 낸다.
data = pd.read_csv("침수심_부산_수정.csv", encoding='CP949')

#기준데이터를 저장할 데이터프레임 만들기
stnd = pd.DataFrame()
stnd['idx'] = data['순번']
stnd['구 별-key1'] = data['행정구역 코드(ADM_CD)'].str.split(" ",expand=True)[1]
stnd['날짜(년-월-일)-key2'] = data['작성 년월(WRITNG_YM)']
stnd['침수심(y)'] = data['침수심(FLUD_DPWT)']
stnd['침수심(y)'].isnull().value_counts()


False    206
True       1
Name: 침수심(y), dtype: int64

In [330]:
stnd['침수심(y)']=stnd['침수심(y)'].fillna(0.001)
stnd


,idx,구 별-key1,날짜(년-월-일)-key2,침수심(y)
0,30581,사하구,2009-12-01,0.43300
1,30583,해운대구,2009-12-01,0.55000
2,30584,해운대구,2009-12-01,0.40000
3,30585,해운대구,2009-12-01,0.45000
4,30586,해운대구,2009-12-01,0.53000
...,...,...,...,...
202,101470,동구,2018-08-01,0.40000
203,101998,연제구,2019-09-01,0.40000
204,101999,강서구,2019-09-01,0.75000
205,102000,강서구,2019-09-01,0.75000


2. 강수량 csv 에서 구별, 강수량 열 데이터를 불러온다.

In [331]:
#부산_일강수량_평균.csv 불러오기
rain = pd.read_csv("부산_일강수량_평균.csv", encoding='CP949')

#null 값 0으로 채우기
rain['강수량(mm)'] = rain['강수량(mm)'].fillna(0)

#필요 없는 열 삭제
rain = rain.drop('지점', axis=1)

#stnd, rain 합치기
#stnd:'날짜(년-월-일)-key2' 와 rain:'날짜' 같은 것끼리 연결
merge_rain = stnd.merge(rain, left_on='날짜(년-월-일)-key2', right_on='날짜')

#'날짜'열 삭제
merge_rain = merge_rain.drop('날짜', axis=1)
#열 이름 바꾸기
merge_rain = merge_rain.rename(columns={'강수량(mm)':'강수량(일 평균)'})
merge_rain

,idx,구 별-key1,날짜(년-월-일)-key2,침수심(y),강수량(일 평균)
0,30581,사하구,2009-12-01,0.43300,0.00000
1,30583,해운대구,2009-12-01,0.55000,0.00000
2,30584,해운대구,2009-12-01,0.40000,0.00000
3,30585,해운대구,2009-12-01,0.45000,0.00000
4,30586,해운대구,2009-12-01,0.53000,0.00000
...,...,...,...,...,...
202,101470,동구,2018-08-01,0.40000,0.00000
203,101998,연제구,2019-09-01,0.40000,2.30000
204,101999,강서구,2019-09-01,0.75000,2.30000
205,102000,강서구,2019-09-01,0.75000,2.30000


3. 하수도 보급률 csv에서 인구, 면적, 구별 열 데이터 불러온다.

4. 하수도 보급률 csv에서 지류, 구 열 뽑기

In [332]:
#하수도 보급률_부산만.csv 불러오기
swer = pd.read_csv("하수도 보급률_부산만.csv", encoding='CP949')

#인구, 면적, 구별 열 데이터 불러온다.
swer = swer[['시도', '구군', '총인구(명)', '총면적']]


#새로운 인구밀도 열 생성 후 (인구/면적) 값 넣기
pd.options.display.float_format = '{:.5f}'.format #지수표현식으로 나오지 않도록 설정
swer['인구밀도'] = swer['총인구(명)'] / swer['총면적']

#인구, 면적, 시도 열 삭제
swer = swer.drop(['총인구(명)', '총면적', '시도'], axis=1)

#구별 , 인구밀도 데이터를 기준데이터에 맞춰 병합   
swer = swer[['구군', '인구밀도']]

#swer, merge_rain 합치기
#swer:'구 별-key1' 와 merge_rain:'시도구군' 같은 것끼리 연결
merge_swer = merge_rain.merge(swer, left_on='구 별-key1', right_on='구군', how = 'outer')

#'구군' 열 삭제
merge_swer = merge_swer.drop('구군', axis=1)

#열 이름 바꾸기
merge_swer = merge_swer.rename(columns={'인구밀도':'인구밀도(인구/면적)'})

#NaN값이 있으면 제거하기
merge_swer = merge_swer.dropna(subset=['구 별-key1'], how='any', axis=0)

merge_swer

,idx,구 별-key1,날짜(년-월-일)-key2,침수심(y),강수량(일 평균),인구밀도(인구/면적)
0,30581.00000,사하구,2009-12-01,0.43300,0.00000,7555.06344
1,40137.00000,사하구,2013-01-01,0.50000,0.00000,7555.06344
2,40138.00000,사하구,2013-01-01,0.55000,0.00000,7555.06344
3,90711.00000,사하구,2017-01-01,0.40000,0.00000,7555.06344
4,90710.00000,사하구,2017-01-01,0.40000,0.00000,7555.06344
...,...,...,...,...,...,...
202,90598.00000,북구,2015-01-01,0.75000,0.00000,7264.46533
203,90591.00000,북구,2015-01-01,0.70000,0.00000,7264.46533
204,90592.00000,북구,2015-01-01,0.60000,0.00000,7264.46533
205,90589.00000,북구,2015-01-01,0.75000,0.00000,7264.46533


5. 하수도_인구_및_보급률 csv에서 보급률과 구 컬럼 추출 후 기준데이터에 병합

In [333]:
#하수도_인구_및_보급률(2009-2020).csv 불러오기
swer2 = pd.read_csv("하수도_인구_및_보급률(2009-2020).csv", encoding='CP949')

#날짜 비교를 위해 년도만 문자열 슬라이싱
merge_swer['년'] = merge_swer['날짜(년-월-일)-key2'].str.slice(start=0, stop=4)
merge_swer['년'] = merge_swer['년'].fillna(0)

#'구군별(1)'과 '하수도 보급률 (%)'만 추출
swer2 = swer2[['시점', '구군별(1)', '하수도 보급률 (%)']]

#swer2, merge_swer 합치기
#swer2:'구군별(1)' 와 merge_swer:'구 별-key1' && swer2: 같은 것끼리 연결
merge_swer['년'] = merge_swer['년'].astype(int)
merge_swer2 = merge_swer.merge(swer2, left_on=['구 별-key1', '년'], right_on=['구군별(1)', '시점'], how = 'outer')

#NaN값이 있으면 제거하기
merge_swer2 = merge_swer2.dropna(subset=['idx'], how='any', axis=0)


#2021년 하수도 보급률 데이터를 2020년 데이터로 채우기
swer2_2021 = swer2[swer2['시점'] == 2020]
swer2_2021['시점'] = swer2_2021['시점'].replace(2020, 2021)

#merge_swer, swer2_2021 합치기
#merge_swer:'구 별-key1' 와 swer2_2021:'구군별(1)' &&  merge_swer:'년'과 swer2_2021:'시점'같은 것끼리 연결
merge_swer2 = merge_swer2.merge(swer2_2021, left_on=['구 별-key1', '년'], right_on=['구군별(1)', '시점'], how = 'outer')

#NaN값이 있으면 제거하기
merge_swer2 = merge_swer2.dropna(subset=['idx'], how='any', axis=0)

#2021년 빈 데이터 값 채우기
merge_swer2['하수도 보급률 (%)_x'] = merge_swer2['하수도 보급률 (%)_x'].fillna(merge_swer2['하수도 보급률 (%)_y'])

#안 쓰는 열 삭제
merge_swer2 = merge_swer2.drop(['년', '시점_x', '시점_y', '구군별(1)_x','구군별(1)_y', '하수도 보급률 (%)_y'], axis=1)
merge_swer2 = merge_swer2.rename(columns={'하수도 보급률 (%)_x':'하수도 보급률 (%)'})
merge_swer2

C:\Users\gmltj\AppData\Local\Temp\ipykernel_12216\528795499.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swer2_2021['시점'] = swer2_2021['시점'].replace(2020, 2021)


,idx,구 별-key1,날짜(년-월-일)-key2,침수심(y),강수량(일 평균),인구밀도(인구/면적),하수도 보급률 (%)
0,30581.00000,사하구,2009-12-01,0.43300,0.00000,7555.06344,100.00000
1,40137.00000,사하구,2013-01-01,0.50000,0.00000,7555.06344,100.00000
2,40138.00000,사하구,2013-01-01,0.55000,0.00000,7555.06344,100.00000
3,90345.00000,사하구,2013-12-01,0.40000,0.00000,7555.06344,100.00000
4,90346.00000,사하구,2013-12-01,0.55000,0.00000,7555.06344,100.00000
...,...,...,...,...,...,...,...
202,90598.00000,북구,2015-01-01,0.75000,0.00000,7264.46533,100.00000
203,90591.00000,북구,2015-01-01,0.70000,0.00000,7264.46533,100.00000
204,90592.00000,북구,2015-01-01,0.60000,0.00000,7264.46533,100.00000
205,90589.00000,북구,2015-01-01,0.75000,0.00000,7264.46533,100.00000


6. 하수도 보급률 csv에서 구 , 설치율 컬럼 추출 후 기준데이터에 병합

In [334]:
#하수도 보급률_부산만.csv 불러오기
swer3 = pd.read_csv("하수도 보급률_부산만.csv", encoding='CP949')
swer3 = swer3[['구군','총면적','하수도설치율','하수처리구역 내 면적']]

swer3['하수도설치율']=swer3['하수도설치율']/swer3['총면적']

merge_swer3 = merge_swer2.merge(swer3, left_on='구 별-key1', right_on='구군', how = 'outer')

#'구군' 열 삭제
merge_swer3 = merge_swer3.drop('구군', axis=1)
merge_swer3 = merge_swer3.drop('총면적', axis=1)

#열 이름 바꾸기
merge_swer3 = merge_swer3.rename(columns={'하수도설치율':'하수도 설치율(%)/총면적'})

#NaN값이 있으면 제거하기
merge_swer3 = merge_swer3.dropna(subset=['idx'], how='any', axis=0)

merge_swer3

,idx,구 별-key1,날짜(년-월-일)-key2,침수심(y),강수량(일 평균),인구밀도(인구/면적),하수도 보급률 (%),하수도 설치율(%)/총면적,하수처리구역 내 면적
0,30581.00000,사하구,2009-12-01,0.43300,0.00000,7555.06344,100.00000,1.76203,22.08800
1,40137.00000,사하구,2013-01-01,0.50000,0.00000,7555.06344,100.00000,1.76203,22.08800
2,40138.00000,사하구,2013-01-01,0.55000,0.00000,7555.06344,100.00000,1.76203,22.08800
3,90345.00000,사하구,2013-12-01,0.40000,0.00000,7555.06344,100.00000,1.76203,22.08800
4,90346.00000,사하구,2013-12-01,0.55000,0.00000,7555.06344,100.00000,1.76203,22.08800
...,...,...,...,...,...,...,...,...,...
202,90598.00000,북구,2015-01-01,0.75000,0.00000,7264.46533,100.00000,1.88214,10.65600
203,90591.00000,북구,2015-01-01,0.70000,0.00000,7264.46533,100.00000,1.88214,10.65600
204,90592.00000,북구,2015-01-01,0.60000,0.00000,7264.46533,100.00000,1.88214,10.65600
205,90589.00000,북구,2015-01-01,0.75000,0.00000,7264.46533,100.00000,1.88214,10.65600


7. 배수 펌프장 서비스 csv 에서 배수면적, 처리능력, 구 데이터 추출

In [335]:
#한국국토정보공사_배수펌프장 서비스_수정.csv 불러오기
p_svc = pd.read_csv("부산광역시_배수 펌프장 현황_20220228.csv", encoding='CP949')

#구군, 배수량(제곱미터_분) 데이터 추출
p_svc = p_svc[['구군', '배수량(제곱미터_분)','유수지 용량(제곱미터)','유수지 면적(제곱미터)']]

p_svc['유수지 용량(제곱미터)']=p_svc['유수지 용량(제곱미터)']/p_svc['유수지 면적(제곱미터)']
#구별 평균 내기
mean = p_svc.groupby('구군')
mean = mean.mean().reset_index() #평균
mean['구군']  = mean['구군'].str.strip()  # 앞 뒤 공백을 제거

#데이터 합치기
#merge_swer3, mean 합치기
#mean:'구' 와 merge_swer3:'구 별-key1': 같은 것끼리 연결
merge_p_svc = merge_swer3.merge(mean, left_on='구 별-key1', right_on='구군', how ='outer')
merge_p_svc
merge_p_svc = merge_p_svc.drop(['구군','유수지 면적(제곱미터)'], axis=1)

#열 이름 바꾸기
merge_p_svc = merge_p_svc.rename(columns={'유수지 용량(제곱미터)':'유수지 면적 내 용량'})

#NaN값이 있으면 제거하기
merge_p_svc = merge_p_svc.dropna(subset=['idx'], how='any', axis=0)

#NaN은 0으로 채우기
merge_p_svc['배수량(제곱미터_분)'] = merge_p_svc['배수량(제곱미터_분)'].fillna(merge_p_svc['배수량(제곱미터_분)'].mean())
merge_p_svc['유수지 면적 내 용량'] = merge_p_svc['유수지 면적 내 용량'].fillna(merge_p_svc['유수지 면적 내 용량'].mean())
merge_p_svc

,idx,구 별-key1,날짜(년-월-일)-key2,침수심(y),강수량(일 평균),인구밀도(인구/면적),하수도 보급률 (%),하수도 설치율(%)/총면적,하수처리구역 내 면적,배수량(제곱미터_분),유수지 면적 내 용량
0,30581.00000,사하구,2009-12-01,0.43300,0.00000,7555.06344,100.00000,1.76203,22.08800,1677.50000,2.42950
1,40137.00000,사하구,2013-01-01,0.50000,0.00000,7555.06344,100.00000,1.76203,22.08800,1677.50000,2.42950
2,40138.00000,사하구,2013-01-01,0.55000,0.00000,7555.06344,100.00000,1.76203,22.08800,1677.50000,2.42950
3,90345.00000,사하구,2013-12-01,0.40000,0.00000,7555.06344,100.00000,1.76203,22.08800,1677.50000,2.42950
4,90346.00000,사하구,2013-12-01,0.55000,0.00000,7555.06344,100.00000,1.76203,22.08800,1677.50000,2.42950
...,...,...,...,...,...,...,...,...,...,...,...
202,90598.00000,북구,2015-01-01,0.75000,0.00000,7264.46533,100.00000,1.88214,10.65600,935.00000,2.00000
203,90591.00000,북구,2015-01-01,0.70000,0.00000,7264.46533,100.00000,1.88214,10.65600,935.00000,2.00000
204,90592.00000,북구,2015-01-01,0.60000,0.00000,7264.46533,100.00000,1.88214,10.65600,935.00000,2.00000
205,90589.00000,북구,2015-01-01,0.75000,0.00000,7264.46533,100.00000,1.88214,10.65600,935.00000,2.00000


8. 태풍 csv에서 날짜, 태풍 이름 컬럼 추출 후 기준 데이터에 합치기

In [336]:
#한국국토정보공사_배수펌프장 서비스_수정.csv 불러오기
typ = pd.read_csv("태풍목록(2009-2021)_수정.csv", encoding='CP949')


#날짜, 태풍 이름 추출
typ = typ[['태풍명', '발생']]

#'발생' 열 가공
typ['발생'] = typ['발생'].str.replace('.', '-')

#merge_swer3, p_svc 합치기
#typ:'발생' 과 merge_p_svc :'날짜(년-월-일)-key2': 같은 것끼리 연결
merge_typ = merge_p_svc .merge(typ, left_on='날짜(년-월-일)-key2', right_on='발생', how = 'outer')

#'발생' 열 삭제
merge_typ = merge_typ.drop('발생', axis=1)

#NaN값이 있으면 제거하기
merge_typ = merge_typ.dropna(subset=['idx'], how='any', axis=0)

#태풍 열 이름 바꾸기 : '태풍명' => '태풍 발생 여부'
merge_typ = merge_typ.rename(columns={'태풍명':'태풍 발생 여부'})

#NAN값을 제외한 태풍 확인
merge_typ['태풍 발생 여부'].dropna()

C:\Users\gmltj\AppData\Local\Temp\ipykernel_12216\2392418999.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  typ['발생'] = typ['발생'].str.replace('.', '-')


169    하구핏
170    하구핏
171    하구핏
172    하구핏
173    하구핏
174    하구핏
175    하구핏
176    하구핏
Name: 태풍 발생 여부, dtype: object

In [337]:
# #merge_typ[merge_typ['태풍 발생 여부'].empty, : ]
# #태풍 이름이 있으면 1로 채우기
# if merge_typ['태풍 발생 여부'] != merge_typ['태풍 발생 여부'].isnull():
#   merge_typ['태풍 발생 여부'] = 0
#태풍 이름이 있으면 1로 채우기
merge_typ['태풍 발생 여부'] = merge_typ['태풍 발생 여부'].replace('하구핏', 1)

#태풍 발생하지 않은 행(NaN)은 0으로 채우기
merge_typ['태풍 발생 여부'] = merge_typ['태풍 발생 여부'].fillna(0)
merge_typ['태풍 발생 여부'] = pd.to_numeric(merge_typ['태풍 발생 여부']) 


In [338]:
import numpy as np

#침수심_부산_수정.csv 불러오기
cause = pd.read_csv("침수심_부산_수정.csv", encoding='CP949')

#순번, 침수 재해 명(F_DSSTR_NM), 침수 원인 상세(F_CUE_DTL) 추출
cause = cause[['순번', '침수 재해 명(F_DSSTR_NM)', '침수 원인 상세(F_CUE_DTL)']]

temp = cause[cause['침수 재해 명(F_DSSTR_NM)'].str.contains('태풍')|cause['침수 원인 상세(F_CUE_DTL)'].str.contains('태풍')]
temp2 = temp['순번']
temp2= temp2.to_frame()
temp2['태풍 여부'] = 1

#merge_swer3, p_svc 합치기
#typ:'발생' 과 merge_p_svc :'날짜(년-월-일)-key2': 같은 것끼리 연결
merge_typ2 = merge_typ.merge(temp2, left_on='idx', right_on='순번', how = 'outer')

#기존에 있던 태풍 발생 여부 데이터랑 합치기 : NaN 값이 있는 곳에 기존 데이터 채워넣기
merge_typ2['temp'] = np.where(pd.notnull(merge_typ2['태풍 여부']) ==True, merge_typ2['태풍 여부'], merge_typ2['태풍 발생 여부'])

#필요 없는 열 삭제
merge_typ2 = merge_typ2.drop(['순번', '태풍 발생 여부', '태풍 여부'], axis=1)
merge_typ2 = merge_typ2.rename(columns={'temp':'태풍 발생 여부'})

#Y:침수심, X1:강수량, X2:인구밀도, X3:하수도보급률, X4:하수도설치율/면적, X5:하수처리구역내면적, X6:배수량, X7:유수지용량, X8:태풍발생여부
merge_typ2=merge_typ2.drop(['idx','구 별-key1','날짜(년-월-일)-key2'],axis=1)
merge_typ2.columns = ['Y','X1','X2','X3','X4','X5','X6','X7','X8']

merge_typ2=merge_typ2.replace(0,0.001)

merge_typ2.to_csv('전처리완료데이터.csv',index=False)
merge_typ2

,Y,X1,X2,X3,X4,X5,X6,X7,X8
0,0.43300,0.00100,7555.06344,100.00000,1.76203,22.08800,1677.50000,2.42950,0.00100
1,0.55000,0.00100,7881.56207,100.00000,1.25704,20.58600,433.42857,4.32099,0.00100
2,0.40000,0.00100,7881.56207,100.00000,1.25704,20.58600,433.42857,4.32099,0.00100
3,0.45000,0.00100,7881.56207,100.00000,1.25704,20.58600,433.42857,4.32099,0.00100
4,0.53000,0.00100,7881.56207,100.00000,1.25704,20.58600,433.42857,4.32099,0.00100
...,...,...,...,...,...,...,...,...,...
202,0.40000,0.00100,9176.69706,100.00000,6.48430,7.73700,220.00000,3.99370,0.00100
203,0.65000,0.00100,3618.36703,100.00000,1.03860,10.61600,590.00000,3.99370,0.00100
204,1.00000,0.00100,10173.59194,100.00000,2.41701,20.55300,146.60000,2.06004,0.00100
205,0.40000,0.00100,784.51705,85.19000,0.29093,54.86500,1133.00000,1.89644,0.00100
